### [Discovering and Exploiting Additive Structure for Bayesian Optimization](https://proceedings.mlr.press/v54/gardner17a.html)


In [ ]:
!pip install GPy

In [2]:
import random
from random import choice

In [3]:
import scipy as sp
from scipy.stats import randint

In [4]:
import numpy as np

In [5]:
import GPy

In [ ]:
'''
def merge(M):
    k1 = np.random.choice(M)
    M.remove(k1)
    k2 = np.random.choice(M)
    M.remove(k2)
    k1.extend(k2)
    M.extend([k1])
    return M

def split(M):
    k1 = np.random.choice(M)
    M.remove(k1)
    l1, l2 = [], []
    for i in k1:
        s = choice([0, 1])
        if s:
            l1.append(i)
        else:
            l2.append(i)
    if len(l1) == 0 or len(l2) == 0:        #if l = [], error 
        M.extend([k1])
    else:
        M.extend([l1, l2])
    return M

def q(M):
    s = choice([0, 1])
    if s:
        print('merge')
        return merge(M)
    else:
        print('split')
        return split(M)
'''

In [6]:
def fix_num(M):
    all_cat = np.unique(M)
    fix = np.arange(1, len(all_cat)+1)
    for i in range(len(all_cat)):
        if fix[i] != all_cat[i]:
            change = np.where(M == all_cat[i])
            M[change] = fix[i]
    return M

In [7]:
def split(M, p):

    #q_pdf = 0.5
    all_cat = np.unique(M)       
    d = len(M)                     	                # if all_cat.shape[0] == d error
    #print('d:', d)
    num_p = len(all_cat)                                #num_p ~ numbewr of partitions
    n_p = np.setdiff1d(np.arange(1,d), all_cat).min()   #n_p ~ new_partition
    #print('n_p', n_p)
    c = np.random.choice(all_cat)                       #c ~ choice             [1, 1, 1, 2, 2, 1, 3, 1, 1, 1] -> [1, 1, 1, 2, 2, 1, 4, 1, 1, 1]  !single group issue 
    #print('choice', c)
    p[0] = p[0]*(1/num_p)                               #p[0] ~ p(M'|M)        
    p[1] = p[1]*(2/((num_p+1)*(num_p)))                 #p[1] ~ p(M|M')
    
    #print('q_pdf after choosing partition', q_pdf)

    part = np.where(M == c)[0]                          #part ~ partition
    #print('p', p)
    p[0] = p[0]*(1/2**(part.shape[0]-1))

    #print('q_pdf after splitting partition', q_pdf)

    #print('M before split:', M)
    for i in part: 
        e = np.random.choice([0, 1])
        if e:
            M[i] = n_p
    #print('M after split:', M) 

    return fix_num(M), p

In [8]:
def merge(M, p):

    
    all_cat = np.unique(M)
    c = np.random.choice(all_cat, size=2, replace=False)    # is len(all_cat) == 1, then error
    c = np.sort(c)
    p[0] = p[0]*(1/len(all_cat))*(1/(len(all_cat)-1))*2
    p[1] = p[1]*(1/(len(all_cat)-1))
    #print('c:', c)

    p1 = np.where(M == c[0])[0]                             # [1, 1, 2, 3, 3, 2, 4, 2, 2, 2] -> [1, 1, 1, 3, 3, 1, 4, 1, 1, 1] !!!
    p2 = np.where(M == c[1])[0]

    p_sum = len(p1) + len(p2)
    #print(p_sum)
    p[1] = p[1]*(1/2**(p_sum-1))
    #print('p:', p1, p2)
    #print('M before:', M)

    M[p2] = M[p1][0]

    #print('M after:', M)

    return fix_num(M), p

In [9]:
def q(M):

    all_cat = np.unique(M)
    d = len(M)
    n_partitions = all_cat.shape[0]

    ch = np.random.choice([0,1])

    if ch:

        if n_partitions == d:
            return M, np.array([0.5, 0.5])
        
        else:
            return split(M, np.array([0.5, 0.5]))
    
    else:

        if n_partitions == 1:
            return M, np.array([0.5, 0.5])
        
        else:
            return merge(M, np.array([0.5, 0.5]))

In [10]:
def create_gp(M, xx, yy):

    k = []
    all_cat = np.unique(M)
    n_partitions = all_cat.shape[0]

    for i in range(n_partitions):
        active_dims = np.where(M == all_cat[i])[0]
        input_dim = active_dims.shape[0]
        k.append(GPy.kern.RBF(input_dim, active_dims=active_dims))

    k_sum = k[0]

    for i in range(1, n_partitions):
        k_sum = k_sum.add(k[i])

    print(k_sum)
    return GPy.models.GPRegression(xx, yy, kernel=k_sum)

$f(x) = \frac{1}{2} \sum (x_{i}^{4} - 16x_{i}^{2} + 5x_{i}) $

In [11]:
def stybtang(x):
    y = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in x[i]:
            y[i] += j**4 - 16*(j**2) + 5*j
        y[i] = y[i]*(1/2)
    return y.reshape(x.shape[0], 1)

In [12]:
xx = np.random.uniform(-4, 4, (10, 10))
yy = stybtang(xx)

In [13]:
M_init = np.ones(10)
m_prev = create_gp(M_init, xx, yy)
M_prev = M_init
for i in range(100):
    #print('M_prev:', M_prev)
    M_new, p = q(M_init)
    m_new = create_gp(M_new, xx, yy)
    m_new.optimize(clear_after_finish=True)
    a = np.exp(m_new.log_likelihood() + np.log(p[1]) - (m_prev.log_likelihood() + np.log(p[0])))
    #print('M new:', M_new)
    #print('a:', a)
    if a > np.random.uniform(0,1):
        M_prev = M_new
        m_prev = m_new
        print(M_prev)

  rbf.         |  value  |  constraints  |  priors
  variance     |    1.0  |      +ve      |        
  lengthscale  |    1.0  |      +ve      |        
  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |    1.0  |      +ve      |        
[2. 2. 1. 2. 2. 1. 2. 1. 1. 2.]
  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |    1.0  |      +ve      |        
  rbf_2.variance     |    1.0  |      +ve      |        
  rbf_2.lengthscale  |    1.0  |      +ve      |        


 /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning:overflow encountered in exp
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[2. 3. 1. 3. 3. 1. 3. 1. 1. 2.]
  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |    1.0  |      +ve      |        
  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |    1.0  |      +ve      |        
  rbf_2.variance     |    1.0  |      +ve      |        
  rbf_2.lengthscale  |    1.0  |      +ve      |        
[3. 2. 1. 3. 2. 1. 3. 1. 1. 2.]
  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |  

In [15]:
print(M_prev)

[2. 4. 2. 3. 1. 1. 1. 3. 2. 3.]


In [16]:
M_truth = np.arange(1, 11)
m_truth = create_gp(M_truth, xx, yy)
m_truth.optimize()

  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |    1.0  |      +ve      |        
  rbf_2.variance     |    1.0  |      +ve      |        
  rbf_2.lengthscale  |    1.0  |      +ve      |        
  rbf_3.variance     |    1.0  |      +ve      |        
  rbf_3.lengthscale  |    1.0  |      +ve      |        
  rbf_4.variance     |    1.0  |      +ve      |        
  rbf_4.lengthscale  |    1.0  |      +ve      |        
  rbf_5.variance     |    1.0  |      +ve      |        
  rbf_5.lengthscale  |    1.0  |      +ve      |        
  rbf_6.variance     |    1.0  |      +ve      |        
  rbf_6.lengthscale  |    1.0  |      +ve      |        
  rbf_7.variance     |    1.0  |      +ve      |        
  rbf_7.lengthscale  |    1.0  |      +ve      |        
  rbf_8.variance     |    1.0  

 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


In [ ]:
m_truth.log_likelihood()

-54.23436012350541

In [17]:
m_pred = create_gp(M_prev, xx, yy)
m_pred.optimize()

  sum.               |  value  |  constraints  |  priors
  rbf.variance       |    1.0  |      +ve      |        
  rbf.lengthscale    |    1.0  |      +ve      |        
  rbf_1.variance     |    1.0  |      +ve      |        
  rbf_1.lengthscale  |    1.0  |      +ve      |        
  rbf_2.variance     |    1.0  |      +ve      |        
  rbf_2.lengthscale  |    1.0  |      +ve      |        
  rbf_3.variance     |    1.0  |      +ve      |        
  rbf_3.lengthscale  |    1.0  |      +ve      |        


 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.7/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


In [18]:
m_pred.log_likelihood()

-51.553858182332306